In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support import expected_conditions as EC  # condiciones esperadas...

from selenium.webdriver import ActionChains as AC   # acciones encadenadas, rollo doble click

from selenium.webdriver.common.keys import Keys  # manejar teclas

import time

import pandas as pd
pd.set_option('display.max_columns', None)   
pd.set_option('display.max_rows', None)

from webdriver_manager.chrome import ChromeDriverManager

PATH=ChromeDriverManager().install()

import warnings
warnings.filterwarnings('ignore')

opciones=Options()

# quita la bandera de ser robot
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)

opciones.headless=False   # si True, no aparece la ventana (headless=no visible)

opciones.add_argument('--start-maximized')         # comienza maximizado

#opciones.add_extension('driver/adblock.crx')       # adblocker

opciones.add_argument('user-data-dir=selenium')    # mantiene las coockies

import Utils
from datetime import date

from decouple import config
user_spoty = config('user_spoty', default='')
pass_spoty = config('pass_spoty', default='')

from joblib import Parallel, delayed

In [ ]:
def extracion_csv(url,fecha):
    driver=webdriver.Chrome(PATH)
    driver.get(url)
        
    time.sleep(2)

    boton_login=driver.find_element(By.XPATH,'//*[@id="__next"]/div/div/main/div[2]/div/header/div/div[2]/a[3]/div[1]').click()
        
    time.sleep(2)
        
    input_mail=driver.find_element(By.XPATH,'//*[@id="login-username"]')
        
    input_mail.send_keys(user_spoty)  # escribe texto

    input_clave=driver.find_element(By.XPATH,'//*[@id="login-password"]')
        
        
    input_clave.send_keys(pass_spoty)
        
    boton_acceso=driver.find_element(By.XPATH,'//*[@id="login-button"]/div[1]/p').click()
        
    time.sleep(4)
    
    cookies=driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]').click()
       
        
        
    driver.get('https://charts.spotify.com/charts/view/regional-global-daily/2017-01-01')

    time.sleep(4)
    
    dias = pd.date_range(start='2017-02-01', end='2017-12-31') 
    for i in dias:
        time.sleep(2)
        driver.get(f'https://charts.spotify.com/charts/view/regional-global-daily/'+i.strftime("%Y-%m-%d"))
        
        time.sleep(3)
        boton_descarga=driver.find_element(By.XPATH,'//*[@id="__next"]/div/div/main/div[2]/div[3]/div/div/a/button')
        
        
        webdriver.ActionChains(driver).move_to_element(boton_descarga ).click(boton_descarga ).perform()
        time.sleep(3)

In [ ]:
def extracion_csv(url):
    driver=webdriver.Chrome(PATH)
    driver.get(url)
        
    time.sleep(2)

    boton_login=driver.find_element(By.XPATH,'//*[@id="__next"]/div/div/main/div[2]/div/header/div/div[2]/a[3]/div[1]').click()
        
    time.sleep(2)
        
    input_mail=driver.find_element(By.XPATH,'//*[@id="login-username"]')
        
    input_mail.send_keys(user_spoty)  # escribe texto

    input_clave=driver.find_element(By.XPATH,'//*[@id="login-password"]')
        
        
    input_clave.send_keys(pass_spoty)
        
    boton_acceso=driver.find_element(By.XPATH,'//*[@id="login-button"]/div[1]/p').click()
        
    time.sleep(4)
    
    cookies=driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]').click()
       
        
        
    driver.get('https://charts.spotify.com/charts/view/regional-global-daily/2017-01-01')

    time.sleep(4)
    
    dias = pd.date_range(start='2018-01-01', end='2022-12-08') 
    for i in dias:
        time.sleep(2)
        driver.get(f'https://charts.spotify.com/charts/view/regional-global-daily/'+i.strftime("%Y-%m-%d"))
        
        time.sleep(3)
        boton_descarga=driver.find_element(By.XPATH,'//*[@id="__next"]/div/div/main/div[2]/div[3]/div/div/a/button')
        
        
        webdriver.ActionChains(driver).move_to_element(boton_descarga ).click(boton_descarga ).perform()
        time.sleep(3)

In [ ]:
extracion_csv('https://charts.spotify.com/charts/view/regional-global-daily/2017-01-01')

In [19]:
def cambiar_fecha(fecha):
    fechas=[]
    uno=pd.read_csv(f'../../../Downloads/regional-global-daily-{fecha}.csv', index_col=False)
    uno['fecha']=fecha
    uno['dif']=fecha
    uno.rename(columns={'rank':'position'}, inplace=True)
    
    for i in range(len(uno.fecha)):
        if uno.previous_rank[i]==-1:
            uno.dif[i]=200-uno.position[i]
        else:
            uno.dif[i]=uno.position[i]-uno.previous_rank[i]
            
        
        
            
    
    
    return uno

In [20]:
from tqdm.notebook import tqdm 
dias = pd.date_range(start='2017-01-01', end='2017-12-31') 
lst_listas=[]

lst_listas=Parallel(n_jobs=-1, verbose=True)(delayed(cambiar_fecha)(d.strftime("%Y-%m-%d")) for d in tqdm(dias))
lista_globa_spoty=pd.concat(lst_listas)
    
lista_globa_spoty=lista_globa_spoty.reindex(columns=['fecha','uri','artist_names','track_name','source','position',
                                   'previous_rank','dif','peak_rank','days_on_chart','streams'])

print('hola')

  0%|          | 0/365 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:   16.1s
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:  1.1min


hola


[Parallel(n_jobs=-1)]: Done 365 out of 365 | elapsed:  2.0min finished


In [22]:
lista_globa_spoty.to_csv('../data/lista2017.csv', index=False,index_label=False)

In [ ]:
def extracion_csv(url):
    driver=webdriver.Chrome(PATH)
    driver.get(url)
        
    time.sleep(2)

    boton_login=driver.find_element(By.XPATH,'//*[@id="__next"]/div/div/main/div[2]/div/header/div/div[2]/a[3]/div[1]').click()
        
    time.sleep(2)
        
    input_mail=driver.find_element(By.XPATH,'//*[@id="login-username"]')
        
    input_mail.send_keys(user_spoty)  # escribe texto

    input_clave=driver.find_element(By.XPATH,'//*[@id="login-password"]')
        
        
    input_clave.send_keys(pass_spoty)
        
    boton_acceso=driver.find_element(By.XPATH,'//*[@id="login-button"]/div[1]/p').click()
        
    time.sleep(4)
    
    cookies=driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]').click()
       
        
        
    driver.get('https://charts.spotify.com/charts/view/regional-global-daily/2017-01-01')

    time.sleep(4)
    
    dias = pd.date_range(start='2018-01-01', end='2022-12-08') 
    for i in dias:
        time.sleep(2)
        driver.get(f'https://charts.spotify.com/charts/view/regional-global-daily/'+i.strftime("%Y-%m-%d"))
        
        time.sleep(3)
        boton_descarga=driver.find_element(By.XPATH,'//*[@id="__next"]/div/div/main/div[2]/div[3]/div/div/a/button')
        
        
        webdriver.ActionChains(driver).move_to_element(boton_descarga ).click(boton_descarga ).perform()
        time.sleep(3)

In [ ]:
def cambiar_fecha(fecha):
    fechas=[]
    uno=pd.read_csv(f'../../../Downloads/regional-global-daily-{fecha}.csv', index_col=False)
    uno['fecha']=fecha
    uno['dif']=fecha
    uno.rename(columns={'rank':'position'}, inplace=True)
    
    for i in range(len(uno.fecha)):
        if uno.previous_rank[i]==-1:
            uno.dif[i]=200-uno.position[i]
        else:
            uno.dif[i]=uno.position[i]-uno.previous_rank[i]
            
        
        
            
    
    
    return uno

In [ ]:
from tqdm.notebook import tqdm 
dias = pd.date_range(start='2018-01-01', end='2018-12-31') 
lst_listas=[]

lst_listas=Parallel(n_jobs=-1, verbose=True)(delayed(cambiar_fecha)(d.strftime("%Y-%m-%d")) for d in tqdm(dias))
lista_globa_spoty=pd.concat(lst_listas)
    
lista_globa_spoty=lista_globa_spoty.reindex(columns=['fecha','uri','artist_names','track_name','source','position',
                                   'previous_rank','dif','peak_rank','days_on_chart','streams'])

In [ ]:
lista_globa_spoty.to_csv('../data/lista2018.csv', index=False,index_label=False)

In [ ]:
from tqdm.notebook import tqdm 
dias = pd.date_range(start='2019-01-01', end='2019-12-31') 
lst_listas=[]

lst_listas=Parallel(n_jobs=-1, verbose=True)(delayed(cambiar_fecha)(d.strftime("%Y-%m-%d")) for d in tqdm(dias))
lista_globa_spoty=pd.concat(lst_listas)
    
lista_globa_spoty=lista_globa_spoty.reindex(columns=['fecha','uri','artist_names','track_name','source','position',
                                   'previous_rank','dif','peak_rank','days_on_chart','streams'])

In [ ]:
lista_globa_spoty.to_csv('../data/lista2019.csv', index=False,index_label=False)

In [ ]:
from tqdm.notebook import tqdm 
dias = pd.date_range(start='2020-01-01', end='2020-12-31') 
lst_listas=[]

lst_listas=Parallel(n_jobs=-1, verbose=True)(delayed(cambiar_fecha)(d.strftime("%Y-%m-%d")) for d in tqdm(dias))
lista_globa_spoty=pd.concat(lst_listas)
    
lista_globa_spoty=lista_globa_spoty.reindex(columns=['fecha','uri','artist_names','track_name','source','position',
                                   'previous_rank','dif','peak_rank','days_on_chart','streams'])

In [ ]:
lista_globa_spoty.to_csv('../data/lista2020.csv', index=False,index_label=False)

In [ ]:
from tqdm.notebook import tqdm 
dias = pd.date_range(start='2021-01-01', end='2021-12-31') 
lst_listas=[]

lst_listas=Parallel(n_jobs=-1, verbose=True)(delayed(cambiar_fecha)(d.strftime("%Y-%m-%d")) for d in tqdm(dias))
lista_globa_spoty=pd.concat(lst_listas)
    
lista_globa_spoty=lista_globa_spoty.reindex(columns=['fecha','uri','artist_names','track_name','source','position',
                                   'previous_rank','dif','peak_rank','days_on_chart','streams'])

In [ ]:
lista_globa_spoty.to_csv('../data/lista2021.csv', index=False,index_label=False)

In [ ]:
from tqdm.notebook import tqdm 
dias = pd.date_range(start='2022-01-01', end='2021-12-08') 
lst_listas=[]

lst_listas=Parallel(n_jobs=-1, verbose=True)(delayed(cambiar_fecha)(d.strftime("%Y-%m-%d")) for d in tqdm(dias))
lista_globa_spoty=pd.concat(lst_listas)
    
lista_globa_spoty=lista_globa_spoty.reindex(columns=['fecha','uri','artist_names','track_name','source','position',
                                   'previous_rank','dif','peak_rank','days_on_chart','streams'])

In [ ]:
lista_globa_spoty.to_csv('../data/lista2022.csv', index=False,index_label=False)

In [ ]:
import re
autores=[]
for autor in lista_globa_spoty.artist_names:
    
    
    palabra=re.findall('[a-zA-Z\s]+',str(autor))
    print(palabra)
    
    for i in range(len(palabra)):
        autores.append(palabra[i])
        
autores2=list(set(autores))

In [ ]:
def extraer_autores(autor):
    driver=webdriver.Chrome(PATH)
    driver.get('https://everynoise.com/engenremap.html')
    
    time.sleep(2)
    
    iframe=driver.find_element(By.XPATH, '/html/body/iframe')

    driver.switch_to.frame(iframe)
    input_autor=driver.find_element(By.XPATH, '/html/body/form/input[1]')
    
    time.sleep(2)
    
    input_autor.send_keys(str(autor))  # escribe texto

    input_autor.send_keys(u'\ue007')  # tecla ENTER

    time.sleep(2)
    
    datos=driver.find_element(By.XPATH,'/html/body/div').text
    
    
    
    
    cadena_final=datos
    
    #cadena_final=pd.DataFrame(cadena_final)
    
    driver.quit()
    
    return cadena_final

In [ ]:
ttt=[]
for i in autores2:
    
    ttt.append(extraer_autores(i))

In [ ]:
uniooon=pd.DataFrame(columns=['autor','estilo'])
for a,e in zip(autores2,ttt):
    uniooon=uniooon.append({'autor':a, 'estilo':e},ignore_index=True)